In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../src")
sys.path.append("../../pyASBC/src")

In [3]:
from fluidvec import *
import pyASBC
asbc = pyASBC.Asbc5Corpus()

In [4]:
sent_iter = asbc.iter_sentences()
from itertools import islice
sents = list(islice(sent_iter, 0, 10))

In [5]:
sent_x = sents[9]

In [6]:
compo_vocab = Vocabulary.load("../data/compo_vocab.pkl")
char_vocab = Vocabulary.load("../data/char_vocab.pkl")
word_vocab = Vocabulary.load("../data/word_vocab.pkl")
len(compo_vocab), len(char_vocab), len(word_vocab)

(3504, 20396, 217350)

In [7]:
import pickle
with open("../data/word_mappings.pkl", "rb") as fin:
    word_mappings = pickle.load(fin)


In [8]:
word_mappings.update({
    "<UNK>": {'compos': [0], 'chars': [0], 'word': 0},
    "<PAD>": {'compos': [1], 'chars': [1], 'word': 1}
})

In [9]:
word_mappings["我"]

{'compos': [8], 'chars': [382], 'word': 230}

In [10]:
sent_iter = asbc.iter_sentences()

In [11]:
sents = list(islice(sent_iter, 0, 10))

In [15]:
make_data_item(sents[0])[0]

('時間', ['：'])

In [16]:
encode_data_item(make_data_item(sents[0])[1])

({'compos': [0], 'chars': [0], 'word': 0},
 [{'compos': [2, 3, 4, 5], 'chars': [2, 3], 'word': 4}])

In [19]:
def make_data_item(sent_x, win=2):
    items = []
    for i in range(len(sent_x)):
        target = ""
        context = []
        for cur in range(i-win, i+win+1):
            if cur < 0 or cur >= len(sent_x):
                context.append("<PAD>")
            elif cur == i:                
                target = sent_x[cur][0]
            else:
                context.append(sent_x[cur][0])
        items.append((target, context))
    return items

def encode_data_item(item_x):
    target, context = item_x
    if target in word_mappings:
        target_idx = word_mappings[target]
    else:
        target_idx = word_mappings["<UNK>"]
    ctx_idxs = [word_mappings.get(x, word_mappings["<UNK>"]) for x in context]        
    return (target_idx, ctx_idxs)

In [20]:
from tqdm.auto import tqdm
from itertools import islice
import numpy as np

rng = np.random.RandomState(12345)
word_idxs = np.arange(len(word_vocab))
data_items = []
counter = 0
sent_iter = asbc.iter_sentences()
permuted = rng.permutation(len(word_vocab))
# sent_iter = islice(sent_iter, 0, 6000)

for sent_i, sent_x in tqdm(enumerate(sent_iter)):
    item_idxs = [encode_data_item(x) for x in make_data_item(sent_x)]
    data_items.extend(item_idxs)
    if (sent_i+1) % 20000 == 0:
        counter = int((sent_i+1)/2e4)
        fout = open(f"../data/train_items/train_items_{counter:03d}.pkl", "wb")
        pickle.dump(data_items, fout)
        data_items = []
        fout.close()
        
if data_items:
    counter = counter + 1
    fout = open(f"../data/train_items/train_items_{counter:03d}.pkl", "wb")
    pickle.dump(data_items, fout)
    fout.close()